In [198]:
from IPython import get_ipython;   
get_ipython().magic('reset -sf')
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
import re
import math
from itertools import count
class text_format:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
data = pd.read_excel(r'C:\Users\Mewtwo\Downloads\RFNSA Scrapper.xlsx')



df = pd.DataFrame(data, columns=['Antenna ID No','Antenna Model','Sector','Height - Phase Centre (m)'
                                 ,'Bearing Degrees (true)',
                                 'Mech Downtilt','Elect Downtilt','System',
                                 'Port Number (Band Power per Port (dBm))','Band Power per Port (dBm)'])
df=df.rename(columns={'Antenna Model': 'Antenna', 'Antenna ID No': 'ID', 'Bearing Degrees (true)': 'Bearing'
                  , 'Mech Downtilt': 'MDT', 'Elect Downtilt': 'EDT', 'Height - Phase Centre (m)': 'Height', 
                   'Port Number (Band Power per Port (dBm))': 'Possible Ports', 
                   'Band Power per Port (dBm)': 'Powers'})

#Extracting Carrier (optus, telstra, Vodafone), technology (LTE, NR, WCDMA) and Frequency information from System 
df['System'] = df['System'].str.replace(' \[', ';', regex=True)
df['System'] = df['System'].str.replace('/', ';', 1, regex=True)
df['Carrier'] = df['System'].str.split(" ;").str[0]

df['tech'] = df['System'].str.split(";").str[1]
#display(df['Carrier'])

df.sort_values(by=['Carrier','ID'], ascending=[True, True], inplace = True)
df['Sector'] = df['Sector'].astype(str).replace('\.0', '', regex=True)

#Reset index of dataframe after sorting
df = df.reset_index(drop=True)
df.index += 1

#display( HTML( df.to_html().replace("\\n","<br><li type='1'>") ) )

df['EDT'] = df['EDT'].str.replace(' to ', '-', regex=True)
df['EDT'] = df['EDT'].str.replace(r'\(.*\)','', regex=True)
df['Powers'] = df['Powers'].str.replace('49', '79.4 W')
df['Powers'] = df['Powers'].str.replace('48', '63.1 W')
df['Powers'] = df['Powers'].str.replace('47', '50.1 W')
df['Powers'] = df['Powers'].str.replace('46', '39.8 W')
df['Powers'] = df['Powers'].str.replace('45', '31.6 W')
df['Powers'] = df['Powers'].str.replace('44', '25.1 W')
df['Powers'] = df['Powers'].str.replace('43', '20 W')
df['Powers'] = df['Powers'].str.replace('42', '15.8 W')
df['Powers'] = df['Powers'].str.replace('37', '5 W')
df['Powers'] = df['Powers'].str.replace('18.91', '0.0778 W')

df['Sector']= [''.join(''.join(map(str, m))+' - Sector ' +f'{l}') for l, m in zip(df['Sector'],df['tech'])]

df.fillna(value = 0,inplace = True)

C:\Users\Mewtwo\anaconda3\lib\site-packages\ipykernel_launcher.py:61: FutureWarning: The default value of regex will change from True to False in a future version.


In [199]:
####Where to add powers?########
Mylist =[]

for i in df['Powers']:
    count = 1
    lister = []
    if isinstance(i, str):
        for j in i:
            if j==';':
                count+=1
            else:
                lister.append(count)
        Mylist.append(lister)
    else:
        Mylist.append(list(str(i))) 
        
    
#display(Mylist)
new_k = []
for elem in Mylist:
    kiterator=[]
    for el in elem:
        if el not in kiterator:
            kiterator.append(el)
    new_k.append(kiterator)
k = new_k

df['WhereToAddPower'] = k
#display(k)

In [200]:
###############Wat should the power be?######
df['Powerss'] = df['Powers'].str.split(";")

itArr2 = []
for index, elm in df['Powerss'].iteritems():
    if isinstance(elm, (list, tuple)):
        elm = [sub.replace('W', '') for sub in elm]
        itArr2.append([x for x in elm if x != ''])
    else:
        itArr2.append([elm])
        
df['UnformatedPowers'] = itArr2


#display(df['UnformatedPowers'])
df['Power (W)'] = ['+'.join(map(str, l)) for l in df['UnformatedPowers']]



In [201]:
####Formatting Powers Column with ports numbers

#itArr=[]
#for index, elm in df['Powers'].iteritems():
#    if isinstance(elm, (list, tuple)):
#        itArr.append([f'{i+1}. {l}' for i, l in enumerate(elm)])
#    else:
#        itArr.append([elm])
#df=df.drop(columns='Powers', axis=1)
#df.insert(2, 'Powers', itArr)
#############################################

# Add these Antennas to prox5

In [202]:
AntennasUnique = list( dict.fromkeys(df['Antenna']) )
print('You need to add these antennas\n\n'+str.join(" \n", AntennasUnique)+' \n\nto PROX5')

You need to add these antennas

SB1-220-MDL 
RRV4-65D-R6 
AEQP 
AENB 
RR2VV-6533D-R6 
AEQE 

to PROX5


In [203]:
IDUnique = list( dict.fromkeys(df['ID'],df['Antenna']) )

# IDs for Antennas

In [204]:
AntennaIds = {}
for antenna in AntennasUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['Antenna'] == antenna, 'ID']))
    AntennaIds[antenna] = IdsForAntenna

for x, y in AntennaIds.items():
    print(f'\nThe id of '+text_format.BOLD+ x + text_format.END + ' are\n'+str.join(" \n", y))


The id of SB1-220-MDL are
A4

The id of RRV4-65D-R6 are
12-O 
22-O 
32-O

The id of AEQP are
14-O 
24-O 
34-O

The id of AENB are
16-O 
26-O 
36-O

The id of RR2VV-6533D-R6 are
11-V 
21-V 
31-V

The id of AEQE are
13-V 
23-V 
33-V


# Antenna Settings

In [205]:
Settings = {}
for i in IDUnique:
    IdsForAntenna = list(dict.fromkeys(df.loc[df['ID'] == i, 'ID']))
    AntennaIds[antenna] = IdsForAntenna

for i in IDUnique:
        print(f'\nFor '+text_format.BOLD+i+text_format.END+f', the Settings are')
        display(df.loc[(df['ID'] == i),['ID','Height','Bearing','MDT']])


For A4, the Settings are


,ID,Height,Bearing,MDT
1,A4,19.0,87,0



For 12-O, the Settings are


,ID,Height,Bearing,MDT
2,12-O,26.1,10,0
3,12-O,26.1,10,0
4,12-O,26.1,10,0
5,12-O,26.1,10,0
6,12-O,26.1,10,0
7,12-O,26.1,10,0
8,12-O,26.1,10,0



For 14-O, the Settings are


...

,ID,Height,Bearing,MDT
42,23-V,27.7,120,1



For 31-V, the Settings are


,ID,Height,Bearing,MDT
43,31-V,26.1,240,0
44,31-V,26.1,240,0
45,31-V,26.1,240,0
46,31-V,26.1,240,0
47,31-V,26.1,240,0
48,31-V,26.1,240,0



For 33-V, the Settings are


,ID,Height,Bearing,MDT
49,33-V,27.7,240,2


# Set Power and Frequency to ports in Prox5

In [206]:
for i in IDUnique:
        print(text_format.BOLD+i)
        display(df.loc[(df['ID'] == i),['ID','tech','Possible Ports','WhereToAddPower','Power (W)']])

A4


,ID,tech,Possible Ports,WhereToAddPower,Power (W)
1,A4,Microwave Link,0,[0],nan


12-O


,ID,tech,Possible Ports,WhereToAddPower,Power (W)
2,12-O,WCDMA900,1;2;3;4,[3],39.8
3,12-O,NR/LTE700,1;2;3;4,"[1, 2, 3, 4]",63.1 +63.1 +63.1 +63.1
4,12-O,NR/LTE900,1;2;3;4,"[1, 2]",63.1 +63.1
5,12-O,LTE1800,5;6;7;8;9;10;11;12,"[7, 8, 11, 12]",50.1 +50.1 +50.1 +50.1
6,12-O,NR/LTE2100,5;6;7;8;9;10;11;12,"[7, 8, 11, 12]",50.1 +50.1 +50.1 +50.1
7,12-O,LTE2600,5;6;7;8;9;10;11;12,"[5, 6, 9, 10]",50.1 +50.1 +50.1 +50.1
8,12-O,LTE2300,5;6;7;8;9;10;11;12,"[5, 6, 9, 10]",31.6 +31.6 +31.6 +31.6


14-O


...

,ID,tech,Possible Ports,WhereToAddPower,Power (W)
42,23-V,NR3500,1;2,[1],50.1


31-V


,ID,tech,Possible Ports,WhereToAddPower,Power (W)
43,31-V,NR/LTE700,1;2;3;4,"[1, 2, 3, 4]",50.1 +50.1 +50.1 +50.1
44,31-V,NR/LTE1800,5;6;7;8;9;10;11;12,"[5, 6, 7, 8, 9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1
45,31-V,NR/LTE850,1;2;3;4,"[1, 2, 3, 4]",50.1 +50.1 +50.1 +50.1
46,31-V,WCDMA900,1;2;3;4,"[3, 4]",50.1 +50.1
47,31-V,NR/LTE2100,5;6;7;8;9;10;11;12,"[5, 6, 7, 8, 9, 10, 11, 12]",50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1 +50.1
48,31-V,NB-IOT900,1;2;3;4,"[3, 4]",15.8 +15.8


33-V


,ID,tech,Possible Ports,WhereToAddPower,Power (W)
49,33-V,NR3500,1;2,[1],50.1


# EMEG Equipment List

In [207]:
display(df[['ID','EDT','Sector','WhereToAddPower','Power (W)']])

,ID,EDT,Sector,WhereToAddPower,Power (W)
1,A4,0,Microwave Link - Sector nan,[0],nan
2,12-O,2-12,WCDMA900 - Sector 1,[3],39.8
3,12-O,2-12,NR/LTE700 - Sector 1,"[1, 2, 3, 4]",63.1 +63.1 +63.1 +63.1
4,12-O,2-12,NR/LTE900 - Sector 1,"[1, 2]",63.1 +63.1
5,12-O,2-12,LTE1800 - Sector 1,"[7, 8, 11, 12]",50.1 +50.1 +50.1 +50.1
6,12-O,2-12,NR/LTE2100 - Sector 1,"[7, 8, 11, 12]",50.1 +50.1 +50.1 +50.1
7,12-O,2-12,LTE2600 - Sector 1,"[5, 6, 9, 10]",50.1 +50.1 +50.1 +50.1
8,12-O,2-12,LTE2300 - Sector 1,"[5, 6, 9, 10]",31.6 +31.6 +31.6 +31.6
9,14-O,6,NR3500 - Sector 1,[1],79.4
10,16-O,7,NR2300 - Sector 1,[1],63.1
